# Notebook Settings

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%run ../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

# Imports

``` ipython
import sys
sys.path.insert(0, '../')

import torch
import gc
import pandas as pd
from time import perf_counter

from src.network import Network
from src.plot_utils import plot_con
from src.decode import decode_bump
from src.utils import clear_cache
```

# Helpers

``` ipython
def convert_seconds(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s
```

``` ipython
import numpy as np
def get_theta(a, b, GM=0, IF_NORM=0):

    u, v = a, b

    if GM:
        v = b - np.dot(b, a) / np.dot(a, a) * a

    if IF_NORM:
        u = a / np.linalg.norm(a)
        v = b / np.linalg.norm(b)

    return np.arctan2(v, u)
```

``` ipython
def get_idx(model):
    ksi = model.PHI0.cpu().detach().numpy()
    print(ksi.shape)

    theta = get_theta(ksi[0], ksi[2], GM=0, IF_NORM=0)
    return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
    ksi = model.PHI0.cpu().detach().numpy()
    return rates @ ksi.T / rates.shape[-1]

```

# Parameters

``` ipython
REPO_ROOT = "/home/leon/models/NeuroTorch"
conf_name = "config_EI.yml"
```

# RNN with torch

## Single Trial

### Model

``` ipython
start = perf_counter()
model = Network(conf_name, REPO_ROOT, VERBOSE=1, DEVICE='cuda', LIVE_FF_UPDATE=1, TASK='dual_rand', seed=0)  
rates = model(RET_FF=1).cpu().numpy()  
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

Ne = model.Na[0].detach().cpu().numpy()
N = model.N_NEURON

print('rates', rates.shape)
```

``` ipython
overlap = get_overlap(model, rates)
print(overlap.shape)

idx = get_idx(model)
rates_ordered = rates[..., idx]

m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
print(torch.cuda.memory_allocated()/100000)
# del model
clear_cache()
print(torch.cuda.memory_allocated()/100000)
```

### Dynamics

``` ipython
ff_input = model.ff_input.cpu().detach().numpy()
print(ff_input.shape)

fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

ax[0].plot(ff_input[0, :, :5])
ax[0].plot(ff_input[0, :, -5:])
ax[0].set_xlabel('Step')
ax[0].set_ylabel('FF Input')

ax[1].imshow(ff_input[0].T, cmap='jet', vmin=0, aspect='auto')
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Neuron #')
ax[1].set_ylim([0, Ne])
plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

r_max = 15

ax[0].imshow(rates[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max, origin='lower')
ax[0].set_ylabel('Neuron #')
ax[0].set_xlabel('Step')

ax[1].imshow(rates_ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
ax[1].set_ylabel('Pref. Location (°)')
ax[1].set_xlabel('Step')
ax[1].set_yticks(np.linspace(0, Ne, 5), np.linspace(360, 0, 5).astype(int))
# ax[0][1].colorbar()

plt.show()
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

ax[0].plot((m1.T/m0.T))
ax[0].set_xlabel('Step')
ax[0].set_ylabel('$\mathcal{F}_1$')

ax[1].plot((phi.T * 180 / np.pi))
ax[1].set_yticks(np.linspace(0, 360, 5).astype(int), np.linspace(0, 360, 5).astype(int))
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Phase (°)')
plt.show()
```

``` ipython

```

## Multiple initializations

### Parameters

``` ipython
seed = np.random.randint(100)
print(seed)
```

### Simulation

``` ipython
LR_COV = [[1.0, 1.0, 0.0, 0.0],
          [0.8, 1.0, 0., 0.0],
          [0.0, 0., 1.0, 0.8],
          [0.0, 0.0, 0.8, 1.0]]

start = perf_counter()
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE='cuda',
                N_BATCH=100, SEED=seed, LR_COV=LR_COV,
                TASK='dual_rand', LIVE_FF_UPDATE=1, DURATION=10)

rates = model(RET_FF=1).detach().cpu().numpy()
end = perf_counter()

print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
print('rates', rates.shape)
```

``` ipython
plt.hist(model.phase.cpu().numpy())
plt.show()
```

### Results

``` ipython
Ne = model.Na[0].detach().cpu().numpy()
N = model.N_NEURON

ff_input = model.ff_input.cpu().detach().numpy()
print(ff_input.shape)

fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

ax[0].plot(ff_input[0, :, :5])
ax[0].plot(ff_input[1, :, :5])
ax[0].set_xlabel('Step')
ax[0].set_ylabel('FF Input')

ax[1].imshow(ff_input[1].T, cmap='jet', vmin=0, vmax= 400, aspect='auto')
ax[1].set_xlabel('Step')
ax[1].set_ylabel('Neuron #')
ax[1].set_ylim([0, Ne])
plt.show()
```

``` ipython
print(rates.mean(-1).shape)
plt.plot(rates.mean(-1).T)
plt.xlabel('Step')
plt.ylabel('Rates (Hz)')
plt.show()
```

``` ipython
idx = get_idx(model)
rates_ordered = rates[..., idx]
print(rates_ordered.shape)
```

``` ipython
overlap = get_overlap(model, rates)
print(overlap.T.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

ax[0].plot(overlap.T[0])
ax[0].set_ylabel('Overlap on $\\xi_1$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(overlap.T[1])
ax[1].set_ylabel('Overlap on $\\xi_2$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(overlap.T[2])
ax[2].set_ylabel('Overlap on $\\xi_3$ (Hz)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
m0, m1, phi = decode_bump(rates_ordered, axis=-1)
print(m0.shape)
```

``` ipython
fig, ax = plt.subplots(1, 3, figsize=[2.2*width, height])

ax[0].plot(m0.T)
#ax[0].set_ylim([0, 360])
#ax[0].set_yticks([0, 90, 180, 270, 360])
ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
ax[0].set_xlabel('Step')

ax[1].plot(m1.T)
# ax[1].set_ylim([0, 360])
# ax[1].set_yticks([0, 90, 180, 270, 360])
ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
ax[1].set_xlabel('Step')

ax[2].plot(phi.T * 180 / np.pi, alpha=.2)
ax[2].set_ylim([0, 360])
ax[2].set_yticks([0, 90, 180, 270, 360])
ax[2].set_ylabel('Phase (°)')
ax[2].set_xlabel('Step')

plt.show()
```

``` ipython
print(rates_ordered.shape)

plt.imshow(rates_ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=10)
plt.ylabel('Pref. Location (°)')
plt.xlabel('Time (au)')
plt.yticks(np.linspace(0, rates_ordered.shape[-1], 5), np.linspace(0, 360, 5).astype(int))
plt.colorbar()
plt.show()
```

``` ipython
print(m0.shape)
x = m1[:, -1]/ m0[:, -1] * np.cos(phi[:, -1])
y = m1[:, -1] / m0[:, -1] * np.sin(phi[:, -1])

fig, ax = plt.subplots(figsize=(height, height))
ax.plot(x, y, 'o')
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
plt.show()
```

``` ipython

```

## Behavior

### Helpers

``` ipython
def run_behavior(conf_name, cov_list, n_ini, seed, device='cuda', **kwargs):
    start = perf_counter()

    rates = []
    ksi = []
    with torch.no_grad():
        for cov in cov_list:

            model = Network(conf_name, REPO_ROOT, DEVICE=device,
                            LR_COV = [[1.0, 0.9, 0.0, 0.0],
                                      [0.9, 1.0, cov, 0.0],
                                      [0.0, cov, 1.0, 0.9],
                                      [0.0, 0.0, 0.9, 1.0]],
                            LIVE_FF_UPDATE=1,
                            VERBOSE=0, SEED=seed, N_BATCH=2*n_ini, **kwargs)

            rates.append(model().cpu().detach().numpy())
            ksi.append(model.PHI0.cpu().detach().numpy())

            model.cpu()
            del model

            gc.collect()
            torch.cuda.empty_cache()

    end = perf_counter()

    print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

    return np.array(rates), np.array(ksi)
```

### Parameters

``` ipython
REPO_ROOT = "/home/leon/models/NeuroTorch"
conf_name = "config_EI.yml"
```

``` ipython
cov_list = np.linspace(0.1, 0, 5)
print(cov_list)
n_ini = 32
seed = np.random.randint(100)
# 34, 53
print(seed)
```

``` ipython
def ret_overlap(rates, ksi):
    rates_ord = np.zeros(rates.shape)
    overlap = []

    for i in range(len(cov_list)):
        theta = get_theta(ksi[i][0], ksi[i][2], GM=0, IF_NORM=0)

        overlap.append(rates[i] @ ksi[i].T / rates.shape[-1])

        index_order = theta.argsort()
        rates_ord[i] = rates[i][..., index_order]

    return np.array(overlap), rates_ord
```

### Single run

``` ipython
I0 = [.1, 0.0, 0.0]
rates, ksi = run_behavior(conf_name, cov_list, n_ini, seed, device='cuda', I0=I0)
```

``` ipython
print(rates.shape)
print(ksi.shape)
```

``` ipython
I0 = [.1, .1, 0.0]
ratesGo, ksiGo = run_behavior(conf_name, cov_list, n_ini, seed, device='cuda', I0=I0)
```

``` ipython
print(ratesGo.shape)
print(ksiGo.shape)
```

``` ipython
overlap1, rates1 = ret_overlap(rates, ksi)
overlap2, rates2 = ret_overlap(ratesGo, ksiGo)
```

``` ipython
print(overlap1.shape)
print(overlap2.shape)
```

``` ipython
# n_ini = 16
fig, ax = plt.subplots(1, 2, figsize=(2*width, height))

ax[0].plot(overlap1[:, :n_ini, -5:, 0].mean((1,2)), '-rs')
ax[0].plot(overlap1[:, n_ini:, -5:, 0].mean((1,2)), '-ro')
ax[0].set_ylabel('Sample Overlap')
ax[0].set_xlabel('Day')
ax[0].set_title('DPA')

ax[1].plot(overlap2[:, :n_ini, -5:, 0].mean((1,2)), '-bs')
ax[1].plot(overlap2[:, n_ini:, -5:, 0].mean((1,2)), '-bo')
ax[1].set_ylabel('Sample Overlap')
ax[1].set_xlabel('Day')
ax[1].set_title('Dual Go')

plt.show()
```

``` ipython
readout1A = overlap1[:, :n_ini, -5:, 0]
readout1B = overlap1[:, n_ini:, -5:, 0]

readout1 = np.stack((readout1A, readout1B))
# print(readout1.shape)

perf1 = (readout1[0]>0).mean((1, 2))
perf1 += (readout1[1]<0).mean((1, 2))

readout2A = overlap2[:, :n_ini, -5:, 0]
readout2B = overlap2[:, n_ini:, -5:, 0]

readout2 = np.stack((readout2A, readout2B))
print((readout2[0]>0).shape)

perf2 = (readout2[0]>0).mean((1, 2))

perf2 += (readout2[1]<0).mean((1, 2))

plt.plot(perf1/2, 'r')
plt.plot(perf2/2, 'b')

plt.ylabel('Performance')
plt.xlabel('Day')
plt.show()
```

``` ipython

```

1.  Performance from phase

    ``` ipython
    def get_perf(rates):
        m0, m1, phi = decode_bump(rates, axis=-1)
        x = m1[..., -1] / m0[..., -1] * np.cos(phi[..., -1])
        performance = (x[: , :n_ini] < 0).mean(1) * 100
        performance += (x[: , n_ini:] > 0).mean(1) * 100

        return performance / 2
    ```

    ``` ipython
    perf1 = get_perf(rates1)
    print(perf1)
    perf2 = get_perf(rates2)
    print(perf2)

    plt.plot(perf1, 'r')
    plt.plot(perf2, 'b')
    plt.ylabel('Performance')
    plt.xlabel('Day')
    plt.show()
    ```

2.  Single

    ``` ipython
    ini = -1
    overlap = overlap2.copy()
    print(overlap1[ini].shape)
    m0, m1, phi = decode_bump(rates2, axis=-1)
    print(m0[ini].shape)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

    ax[0].plot(overlap[ini, ..., 0].T, alpha=.2)
    ax[0].set_ylabel('Overlap on $\\xi_1$ (Hz)')
    ax[0].set_xlabel('Step')

    ax[1].plot(overlap[ini, ..., 1].T, alpha=.2)
    ax[1].set_ylabel('Overlap on $\\xi_2$ (Hz)')
    ax[1].set_xlabel('Step')

    ax[2].plot(overlap[ini, ..., 2].T, alpha=.2)
    ax[2].set_ylabel('Overlap on $\\xi_3$ (Hz)')
    ax[2].set_xlabel('Step')

    plt.show()
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 3, figsize=[2*width, height])

    ax[0].plot(m0[ini].T)
    #ax[0].set_ylim([0, 360])
    #ax[0].set_yticks([0, 90, 180, 270, 360])
    ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
    ax[0].set_xlabel('Step')

    ax[1].plot(m1[ini].T)
    # ax[1].set_ylim([0, 360])
    # ax[1].set_yticks([0, 90, 180, 270, 360])
    ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
    ax[1].set_xlabel('Step')

    ax[2].plot(phi[ini].T * 180 / np.pi)
    ax[2].set_ylim([0, 360])
    ax[2].set_yticks([0, 90, 180, 270, 360])
    ax[2].set_ylabel('Phase (°)')
    ax[2].set_xlabel('Step')

    plt.show()
    ```

    ``` ipython
    x = m1[ini, ..., -1] / m0[ini, ..., -1] * np.cos(phi[ini, ..., -1])
    y = m1[ini, ..., -1] / m0[ini, ..., -1] * np.sin(phi[ini, ..., -1])

    fig, ax = plt.subplots(figsize=(height, height))
    ax.plot(x.T, y.T, 'o')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    plt.show()
    ```

    ``` ipython

    ```

## Run

``` ipython
I0 = [.1, 0.0, 0.0]
rates_list = []
ksi_list = []

for i in range(10):
    rates, ksi = run_behavior(conf_name, cov_list, n_ini, seed=i, device='cuda', I0=I0)
    rates_list.append(rates)
    ksi_list.append(ksi)

rates_list = np.array(rates_list)
ksi_list = np.array(ksi_list)
```

``` ipython
overlap_list, rate_list = [], []
for i in range(rates_list.shape[0]):
    overlap, rates = ret_overlap(rates_list[i], ksi_list[i])
    rate_list.append(rates)
    overlap_list.append(overlap)

rate_list = np.array(rate_list)
overlap_list = np.array(overlap_list)
```

``` ipython
print(overlap_list.shape)
```

``` ipython
readoutA = overlap_list[..., :n_ini, -5:, 0]
readoutB = overlap_list[..., n_ini:, -5:, 0]

readout = np.stack((readoutA, readoutB))
print(readout.shape)
```

``` ipython
perf = (readout[0]>0).mean((2, 3))
perf += (readout[1]<0).mean((2, 3))

print(perf.shape)

plt.plot(perf.T/2)
plt.show()
```

``` ipython
print(rate_list.shape)
```

``` ipython
m0, m1, phi = decode_bump(rate_list, axis=-1)
print(m0.shape)
```

``` ipython
ini = 0

x = m1[:, ini, ..., -1] / m0[:, ini, ..., -1] * np.cos(phi[:, ini, ..., -1])
y = m1[:, ini, ..., -1] / m0[:, ini, ..., -1] * np.sin(phi[:, ini, ..., -1])

fig, ax = plt.subplots(figsize=(height, height))
ax.plot(x.T, y.T, 'o')
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
plt.show()
```

## Different Realizations

### Helpers

``` ipython
def run_X(conf_name, real_list, n_ini, device='cuda', **kwargs):
    start = perf_counter()

    rates = []
    ksi = []
    with torch.no_grad():
        for real in real_list:

            model = Network(conf_name,
                            REPO_ROOT, DEVICE=device,  VERBOSE=0, SEED=0,
                            LIVE_FF_UPDATE=1, N_BATCH=n_ini, **kwargs)

            # model.I0[0] = .1
            # sample_A = model.init_ff_input()

            # model.I0[0] = -.1
            # sample_B = model.init_ff_input()

            # ff_input = torch.cat((sample_A, sample_B))
            ff_input = None
            ksi.append(model.PHI0.cpu().detach().numpy())
            rates.append(model(ff_input, REC_LAST_ONLY=1).cpu().detach().numpy())

            del model

    end = perf_counter()

    print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

    return np.array(rates), np.array(ksi)
```

### Parameters

``` ipython
REPO_ROOT = "/home/leon/models/NeuroTorch"
conf_name = "config_EI.yml"
```

``` ipython
real_list = np.arange(0, 10)
n_ini = 32
```

### Run

1.  Orthogonal

    ``` ipython
    rates, ksi = run_X(conf_name, real_list, n_ini,
                       device='cuda',
                       LR_COV=[[1.0, 0.9, 0.0, 0.0],
                               [0.9, 1.0, 0.0, 0.0],
                               [0.0, 0.0, 1.0, 0.9],
                               [0.0, 0.0, 0.9, 1.0]]
                       )
    ```

    ``` ipython
    rates_ord = np.zeros(rates.shape)

    for i in real_list:
        idx = np.arange(0, len(ksi[i][0]))
        theta = get_theta(ksi[i][0], ksi[i][2], GM=0, IF_NORM=1)

        index_order = theta.argsort()
        rates_ord[i] = rates[i][ ..., index_order]
    print(rates_ord.shape)

    m0, m1, phi = decode_bump(rates_ord, axis=-1)
    print(m0.shape)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=[2*height, height])

    x = m1 / m0 * np.cos(phi)
    y = m1 / m0 * np.sin(phi)

    ax[0].hist(np.hstack(phi) * 180 / np.pi, density=True, bins='auto')
    ax[0].set_title('$h_S . \\xi_D = 0$')
    ax[0].set_xlim([0, 360])
    ax[0].set_xticks([0, 180, 360])
    ax[0].set_xlabel('Pref. Location (°)')
    ax[0].set_ylabel('Density')

    ax[1].plot(x.T, y.T, 'o')
    ax[1].set_xlim([-2, 2])
    ax[1].set_ylim([-2, 2])
    ax[1].set_title('$h_S . \\xi_D = 0$')
    ax[1].set_xlabel('Sample Axis')
    ax[1].set_ylabel('Distractor Axis')

    plt.show()
    ```

    ``` ipython

    ```

2.  xi<sub>S</sub> . xi<sub>D</sub> \>0

    ``` ipython
    rates_cov, ksi_cov = run_X(conf_name, real_list, n_ini,
                               device='cuda',
                               LR_COV=[[1.0, 0.9, 0.0, 0.0],
                                       [0.9, 1.0, 0.1, 0.0],
                                       [0.0, 0.1, 1.0, 0.9],
                                       [0.0, 0.0, 0.9, 1.0]]
                               )
    ```

    ``` ipython
    rates_ord_cov = np.zeros(rates_cov.shape)

    for i in real_list:
        idx = np.arange(0, len(ksi_cov[i][0]))
        theta = get_theta(ksi_cov[i][0], ksi_cov[i][2], GM=0, IF_NORM=1)

        index_order = theta.argsort()
        rates_ord_cov[i] = rates_cov[i][..., index_order]

    print(rates_ord_cov.shape)

    m0_cov, m1_cov, phi_cov = decode_bump(rates_ord_cov, axis=-1)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=[2*height, height])

    x = m1_cov / m0_cov * np.cos(phi_cov)
    y = m1_cov / m0_cov * np.sin(phi_cov)

    ax[0].hist(np.hstack(phi_cov) * 180 / np.pi, density=True, bins='auto')
    ax[0].set_title('$\\xi_S . \\xi_D > 0$')
    ax[0].set_xlim([0, 360])
    ax[0].set_xticks([0, 180, 360])
    ax[0].set_xlabel('Pref. Location (°)')
    ax[0].set_ylabel('Density')

    ax[1].plot(x.T, y.T, 'o')
    ax[1].set_xlim([-2, 2])
    ax[1].set_ylim([-2, 2])
    # ax[1].set_title('$\\xi_S . \\xi_D > 0$')
    ax[1].set_xlabel('Sample Axis')
    ax[1].set_ylabel('Distractor Axis')

    plt.show()
    ```

    ``` ipython

    ```

3.  h<sub>S</sub> . xi<sub>D</sub> \>0

    ``` ipython
    rates_cov2, ksi_cov2 = run_X(conf_name, real_list, n_ini,
                                 device='cuda',
                                 LR_COV=[[1.0, 0.9, 0.0, 0.0],
                                         [0.9, 1.0, 0.0, 0.0],
                                         [0.0, 0.0, 0.5, 0.4],
                                         [0.0, 0.0, 0.4, 1.0]]
                                 )
    ```

    ``` ipython
    rates_ord_cov2 = np.zeros(rates_cov2.shape)

    for i in real_list:
        idx = np.arange(0, len(ksi_cov2[i][0]))
        theta = get_theta(ksi_cov2[i][0], ksi_cov2[i][2], GM=0, IF_NORM=0)

        index_order = theta.argsort()
        rates_ord_cov2[i] = rates_cov2[i][..., index_order]

    print(rates_ord_cov2.shape)

    m0_cov2, m1_cov2, phi_cov2 = decode_bump(rates_ord_cov2, axis=-1)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=[2*height, height])

    x = m1_cov2 / m0_cov2 * np.cos(phi_cov2)
    y = m1_cov2 / m0_cov2 * np.sin(phi_cov2)

    ax[0].hist(np.hstack(phi_cov2) * 180 / np.pi, density=True, bins=20)
    ax[0].set_title('$h_S . \\xi_D > 0$')
    ax[0].set_xlim([0, 360])
    ax[0].set_xticks([0, 180, 360])
    ax[0].set_xlabel('Pref. Location (°)')
    ax[0].set_ylabel('Density')

    ax[1].plot(x.T, y.T, 'o')
    ax[1].set_xlim([-2, 2])
    ax[1].set_ylim([-2, 2])
    ax[1].set_title('$h_S . \\xi_D > 0$')
    ax[1].set_xlabel('Sample Axis')
    ax[1].set_ylabel('Distractor Axis')

    plt.show()
    ```

    ``` ipython

    ```

4.  h<sub>S</sub> . xi<sub>D</sub> \>0 and xi<sub>S</sub> .
    xi<sub>D</sub>\>0

    ``` ipython
    rates_cov3, ksi_cov3 = run_X(conf_name, real_list, n_ini,
                                 device='cuda',
                                 LR_COV=[[1.0, 0.8, 0.2, 0.0],
                                         [0.8, 1.0, 0.2, 0.0],
                                         [0.2, 0.2, 1.0, 0.8],
                                         [0.0, 0.0, 0.8, 1.0]]
                                 )
    ```

    ``` ipython
    rates_ord_cov3 = np.zeros(rates_cov3.shape)

    for i in real_list:
        idx = np.arange(0, len(ksi_cov3[i][0]))
        theta = get_theta(ksi_cov3[i][0], ksi_cov3[i][2], GM=0, IF_NORM=0)

        index_order = theta.argsort()
        rates_ord_cov3[i] = rates_cov3[i][..., index_order]

    print(rates_ord_cov3.shape)

    m0_cov3, m1_cov3, phi_cov3 = decode_bump(rates_ord_cov3, axis=-1)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=[2*height, height])

    x = m1_cov3 / m0_cov3 * np.cos(phi_cov3)
    y = m1_cov3 / m0_cov3 * np.sin(phi_cov3)

    # fig.suptitle('$\\xi_S . \\xi_D > 0$, $h_S . \\xi_D > 0$', fontsize=22)

    ax[0].hist(np.hstack(phi_cov3) * 180 / np.pi, density=True, bins='auto')
    ax[0].set_title('$\\xi_S . \\xi_D > 0$ and $h_S . \\xi_D > 0$')
    ax[0].set_xlim([0, 360])
    ax[0].set_xticks([0, 180, 360])
    ax[0].set_xlabel('Pref. Location (°)')
    ax[0].set_ylabel('Density')

    ax[1].plot(x, y, 'o')
    ax[1].set_xlim([-2, 2])
    ax[1].set_ylim([-2, 2])
    # ax[1].set_title('$\\xi_S . \\xi_D > 0$ \n $h_S . \\xi_D > 0$')
    ax[1].set_xlabel('Sample Axis')
    ax[1].set_ylabel('Distractor Axis')

    plt.show()
    ```

    ``` ipython

    ```

5.  h<sub>S</sub> . xi<sub>D</sub> \>0 and xi<sub>S</sub> .
    xi<sub>D</sub>\>0

    ``` ipython
    rates_cov4, ksi_cov4 = run_X(conf_name, real_list, n_ini,
                                 device='cuda',
                                 LR_COV=[[1.0, 0.9, 0.0, 0.0],
                                         [0.9, 1.0, 0.0, 0.0],
                                         [0.0, 0.0, 1.0, 0.2],
                                         [0.0, 0.0, 0.2, 1.0]]
                                 )
    ```

    ``` ipython
    rates_ord_cov4 = np.zeros(rates_cov4.shape)

    for i in real_list:
        idx = np.arange(0, len(ksi_cov4[i][0]))
        theta = get_theta(ksi_cov4[i][0], ksi_cov4[i][2], GM=0, IF_NORM=0)

        index_order = theta.argsort()
        rates_ord_cov4[i] = rates_cov4[i][..., index_order]

    print(rates_ord_cov4.shape)

    m0_cov4, m1_cov4, phi_cov4 = decode_bump(rates_ord_cov4, axis=-1)
    ```

    ``` ipython
    fig, ax = plt.subplots(1, 2, figsize=[2*height, height])

    x = m1_cov4 / m0_cov4 * np.cos(phi_cov4)
    y = m1_cov4 / m0_cov4 * np.sin(phi_cov4)

    # fig.suptitle('$\\xi_S . \\xi_D > 0$, $h_S . \\xi_D > 0$', fontsize=22)

    ax[0].hist(np.hstack(phi_cov4) * 180 / np.pi, density=True, bins='auto')
    ax[0].set_title('$\\xi_S . \\xi_D > 0$ and $h_S . \\xi_D > 0$')
    ax[0].set_xlim([0, 360])
    ax[0].set_xticks([0, 180, 360])
    ax[0].set_xlabel('Pref. Location (°)')
    ax[0].set_ylabel('Density')

    ax[1].plot(x.T, y.T, 'o')
    ax[1].set_xlim([-2, 2])
    ax[1].set_ylim([-2, 2])
    # ax[1].set_title('$\\xi_S . \\xi_D > 0$ \n $h_S . \\xi_D > 0$')
    ax[1].set_xlabel('Sample Axis')
    ax[1].set_ylabel('Distractor Axis')

    plt.show()
    ```

    ``` ipython

    ```